# Fichero de permisos

## Origen
Fichero obtenido de la DGT
## Tratamiento
El fichero contiene la cantidad de permisos emitidos por año y por tramo de edad.

<code>
FECHA CENSO,ANYO_EXPEDICION,MUNICIPIO,SEXO,GRUPO PERMISOS,HASTA 18,DE 19 A 24,.....,DE 65 A 74,MÁS DE 74,Total
11/03/2019 ,2000           ,Madrid   ,M   ,B             ,0       ,1         ,.....,64.705    ,14.588   ,358610
11/03/2019 ,2000           ,Madrid   ,M   ,"B,A,A2"      ,0       ,0         ,.....,1.561     ,4.410    ,13612
...
11/03/2019 ,2001           ,Madrid   ,M   ,B             ,0       ,0         ,.... ,126       ,11       ,13135
11/03/2019 ,2001           ,Madrid   ,M   ,"B,A,A2"      ,0       ,0         ,.... ,0         ,0        ,158
</code>
Para poder tratarlo tenemos que desglosar los tipos de permisos e ir acumulando los emitidos por año para tener
el total de permisos vigentes en cada año


In [1]:
import csv
import pandas as pd
import numpy as np

In [2]:
"""
Leemos el fichero csv y lo cargamos en un diccionario por sexo, permiso y año
en la propia lectura separamos los grupos de permisos
"""

acum = {}
with open ('data/permisosMadrid.csv',encoding="ISO-8859-1") as permisosIn:
        reader = csv.DictReader(permisosIn)
        for row in reader:
            for permiso in row['GRUPO PERMISOS'].split(','):
                if not '+E' in permiso and len(permiso.strip())>0: #Algunos terminan en ',' y generan permiso vacío
                    row['GRUPO PERMISOS']=permiso
                    anyo = row['ANYO_EXPEDICION']
                    total = int(row['Total'])
                    sexo = row['SEXO']

                    if not sexo in acum:
                        acum[sexo]={}
                        #print ('acum {}'.format(sexo))
                    if not permiso in acum[sexo]:
                        acum[sexo][permiso]={}
                        #print ('acum {} {}'.format(sexo, anyo))
                    if not anyo in acum[sexo][permiso]:
                        acum[sexo][permiso][anyo]=total
                    else:
                        acum [sexo][permiso][anyo] += total
                        #print ('--> total {} acum {}'.format(total, acum[sexo][anyo][permiso]))

permisosIn.close()


In [3]:
"""
Tratamos el diccionario para ir sumando los valores de los sucesivos años y tener
los vigentes en cada año en lugar de los emitidos.
"""

for sexo in [*acum]:
    for permiso in [*acum[sexo]]:
        for anyo in [*acum[sexo][permiso]]:
            if anyo == '2000':
                tot = acum [sexo][permiso][anyo]
            else:
                acum [sexo][permiso][anyo] += tot
                tot = acum [sexo][permiso][anyo]


In [4]:
"""
Grabamos el diccionario en un fichero csv para poder tratarlo en Tableau
"""

with open('data/permisosAcumulados.csv','w') as f:
    w = csv.DictWriter(f, ['sexo','permiso','anyo','total'])
    w.writeheader()
    for sexo in [*acum]:
        for permiso in [*acum[sexo]]:
            for anyo in [*acum[sexo][permiso]]:
                w.writerow ({'sexo':sexo,'permiso':permiso,'anyo':anyo,'total':acum[sexo][permiso][anyo]})
